In [1]:
!pip install -q tqdm boto3 requests regex sentencepiece sacremoses

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.2 requires botocore<1.34.35,>=1.33.2, but you have botocore 1.29.165 which is incompatible.


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.
bertmodel = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased').to(device)
# Download model and configuration from S3 and cache.

/opt/conda/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


RuntimeError: Missing dependencies: sacremoses

In [3]:
import pandas as pd
df = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/train.csv')
dftest = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/dev.csv')
dfval = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/test.csv')

In [4]:
zerocolumns = df[df['claim']==0]
onecolumns = df[df['claim']==1].sample(frac=1)[:len(zerocolumns)]
dfequal = pd.concat([zerocolumns, onecolumns]).sample(frac=1, random_state=42).reset_index(drop=True)

In [39]:
x_train = []
y_train = []

with torch.no_grad():
    for i in range(len(dfequal)):
    # for i in range(100):
        tweet = dfequal['tweet_text'][i]
        tokens = tokenizer.encode(tweet, add_special_tokens=True)
        tokens_tensor = torch.tensor([tokens]).to(device)
        clsembed = bertmodel(tokens_tensor)[0][0][0]
        x_train.append(clsembed)
        y_train.append(dfequal['claim'][i])
    

In [52]:
x_train_comp = []
y_train_comp = []

with torch.no_grad():
    for i in range(2500):
    # for i in range(100):
        tweet = df['tweet_text'][i]
        tokens = tokenizer.encode(tweet, add_special_tokens=True)
        tokens_tensor = torch.tensor([tokens]).to(device)
        clsembed = bertmodel(tokens_tensor)[0][0][0]
        x_train_comp.append(clsembed)
        y_train_comp.append(df['claim'][i])
    

In [40]:
x_test = []
y_test = []

with torch.no_grad():
    for i in range(len(dftest)):
    # for i in range(100):
        tweet = dftest['tweet_text'][i]
        tokens = tokenizer.encode(tweet, add_special_tokens=True)
        tokens_tensor = torch.tensor([tokens]).to(device)
        clsembed = bertmodel(tokens_tensor)[0][0][0]
        x_test.append(clsembed)
        y_test.append(dftest['claim'][i])

In [6]:
import torch.nn as nn
import torch.optim as optim
import numpy as np

class BertClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BertClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax()
#         self.dropout = nn.Dropout(p=0.3)
#         self.bn1 = nn.BatchNorm1d(input_size)
    
    def forward(self, x):
#         x = self.bn1(x)
        x = self.fc1(x)
        x = self.relu(x)
#         x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
    
model = BertClassifier(768, 256, 2).to(device)

input_size = 768
hidden_size = 256
output_size = 1
learning_rate = 0.0005
num_epochs = 10
batch_size = 32

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# from torch.utils.data import DataLoader

# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [7]:
model

BertClassifier(
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (softmax): Softmax(dim=None)
)

In [51]:
import torch.autograd
from sklearn.metrics import f1_score
# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)

for i in range(num_epochs):
    print(i)
    for i in range(0, len(x_train), batch_size):
#         tweet = df['tweet_text'][i:i+batch_size]
#         tokens = tokenizer.encode(tweet, add_special_tokens=True)
#         tokens_tensor = torch.tensor([tokens]).to(device)
#         clsembed = bertmodel(tokens_tensor)[0][0][0]
#         print(clsembed)
        inputs = torch.stack(x_train[i:i+batch_size]).to(device)
        targets = torch.tensor(y_train[i:i+batch_size]).to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)
    
    with torch.no_grad():
    # Forward pass
        outputs = model(torch.stack(x_train).to(device))

    predicted_labels = torch.argmax(outputs, axis=1)
    print(sum(predicted_labels==0))
    accuracy = f1_score(predicted_labels.cpu().numpy(), y_train)
    print(f"Accuracy on train={accuracy}")
    
    with torch.no_grad():
    # Forward pass on test
        outputs = model(torch.stack(x_test).to(device))

    predicted_labels = torch.argmax(outputs, axis=1)
    print(sum(predicted_labels==0))
    y_actual = torch.tensor(y_test).to(device)
    accuracy = f1_score(predicted_labels.cpu().numpy(), y_test)
    print(f"Accuracy on test={accuracy}")
    

0


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(0.6812, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(740, device='cuda:0')
Accuracy on train=0.6726315789473685
tensor(616, device='cuda:0')
Accuracy on test=0.7534372135655363
1


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(0.6525, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(764, device='cuda:0')
Accuracy on train=0.6940298507462686
tensor(630, device='cuda:0')
Accuracy on test=0.7518450184501846
2


...

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(0.4876, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(874, device='cuda:0')
Accuracy on train=0.8267270668176671
tensor(693, device='cuda:0')
Accuracy on test=0.7287410926365794
19


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(0.4978, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(868, device='cuda:0')
Accuracy on train=0.8442437923250564
tensor(683, device='cuda:0')
Accuracy on test=0.7328605200945626


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [53]:
# Testing accuracy
with torch.no_grad():
    # Forward pass
    outputs = model(torch.stack(x_train).to(device))

predicted_labels = torch.argmax(outputs, axis=1)
print(sum(predicted_labels==0))
accuracy = (predicted_labels == torch.tensor(y_train).to(device)).float().mean()
print(f"Accuracy={accuracy}")

tensor(872, device='cuda:0')
Accuracy=0.8352272510528564


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [62]:
# from torcheval.metrics.functional import multiclass_f1_score

with torch.no_grad():
    # Forward pass
    outputs = model(torch.stack(x_test).to(device))

predicted_labels = torch.argmax(outputs, axis=1)
print(sum(predicted_labels==0))
y_actual = torch.tensor(y_test).to(device)
accuracy = (predicted_labels == y_actual).float().mean()
print(f"Accuracy={accuracy}")



tensor(347, device='cuda:0')
Accuracy=0.7020000219345093


In [32]:
cur_model = model

df_submit = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/submission_format.csv')
df_submit = df_submit[df_submit['task']==1]
x_submit = []
y_submit = []

with torch.no_grad():
    for i in range(len(df_submit)):
    # for i in range(100):
        tweet = df_submit['text'][i]
        tokens = tokenizer.encode(tweet, add_special_tokens=True)
        tokens_tensor = torch.tensor([tokens]).to(device)
        clsembed = bertmodel(tokens_tensor)[0][0][0]
        x_submit.append(clsembed)
        y_submit.append(df_submit['claim'][i])


In [33]:
with torch.no_grad():
    # Forward pass
    outputs = model(torch.stack(x_submit).to(device))
predicted_labels = torch.argmax(outputs, axis=1)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [34]:
df_te = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/submission_format.csv')
labels = predicted_labels.to('cpu')
# df_te[df_te['task']==1]['claim'] = labels.numpy().copy()
df_te.loc[df_te['task']==1, 'claim'] = labels.numpy()

In [35]:
df_te.to_csv('ans.csv')

In [36]:
df_te[df_te['task']==1]['claim']

0       1
1       0
2       1
3       0
4       1
       ..
1493    1
1494    0
1495    1
1496    1
1497    0
Name: claim, Length: 1498, dtype: int64